In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSPublicCoverageDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'ADB_acs_pubcov'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'ADB'
FAIR_INTERVENTION_PARAMS_LST = ['debiased_classifier']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  bf843ff8-62e9-4aac-83bc-d805e3299fdc


## Initialize custom objects

In [9]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,MAR,SEX,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,ESR,ST,FER,RAC1P,AGEP,PINCP
0,19,5,1,2,0,1,3,4,1,1,2,2,2,6,6,0,1,21,3150.0
1,16,5,1,2,0,3,3,4,4,1,2,2,2,1,6,0,9,18,1600.0
2,13,5,2,2,1,1,1,0,2,1,2,2,2,6,6,2,1,16,0.0
3,20,1,2,2,0,4,1,4,1,2,2,2,2,6,6,2,8,43,0.0
4,16,1,2,2,0,4,1,4,1,2,2,2,2,6,6,0,6,54,0.0


In [10]:
data_loader.X_data.shape

(15000, 19)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 14:47:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 100,
 'session_uuid': '84eeb5f0-4ebe-4d9f-94ef-53ae302c2264'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 14:47:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5', 'cat__RELP_6',
       'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP', 'num__WKHP',
       'SEX&RAC1P_binary'],
      dtype='object', length=724)
Top indexes of an X_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype

...

epoch 0; iter: 0; batch classifier loss: 0.709423; batch adversarial loss: 0.826485
epoch 1; iter: 0; batch classifier loss: 0.589269; batch adversarial loss: 0.774333
epoch 2; iter: 0; batch classifier loss: 0.757826; batch adversarial loss: 0.757554
epoch 3; iter: 0; batch classifier loss: 0.651808; batch adversarial loss: 0.683050
epoch 4; iter: 0; batch classifier loss: 0.559073; batch adversarial loss: 0.616526
epoch 5; iter: 0; batch classifier loss: 0.383718; batch adversarial loss: 0.587390
epoch 6; iter: 0; batch classifier loss: 0.361682; batch adversarial loss: 0.608442
epoch 7; iter: 0; batch classifier loss: 0.300812; batch adversarial loss: 0.533616
epoch 8; iter: 0; batch classifier loss: 0.428041; batch adversarial loss: 0.522132
epoch 9; iter: 0; batch classifier loss: 0.334713; batch adversarial loss: 0.554177
epoch 10; iter: 0; batch classifier loss: 0.337976; batch adversarial loss: 0.478305
epoch 11; iter: 0; batch classifier loss: 0.403465; batch adversarial loss:

2024-01-08 14:49:25.594013: W tensorflow/c/c_api.cc:304] Operation '{name:'04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign' id:3205 op device:{requested: '', assigned: ''} def:{{{node 04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1, 04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.683073; batch adversarial loss: 0.620856
epoch 1; iter: 0; batch classifier loss: 0.440653; batch adversarial loss: 0.621911
epoch 2; iter: 0; batch classifier loss: 0.365344; batch adversarial loss: 0.600834
epoch 3; iter: 0; batch classifier loss: 0.300769; batch adversarial loss: 0.601682
epoch 4; iter: 0; batch classifier loss: 0.272899; batch adversarial loss: 0.534356
epoch 5; iter: 0; batch classifier loss: 0.387933; batch adversarial loss: 0.513569
epoch 6; iter: 0; batch classifier loss: 0.376173; batch adversarial loss: 0.530133
epoch 7; iter: 0; batch classifier loss: 0.369582; batch adversarial loss: 0.561520
epoch 8; iter: 0; batch classifier loss: 0.275634; batch adversarial loss: 0.532561
epoch 9; iter: 0; batch classifier loss: 0.282988; batch adversarial loss: 0.506175
epoch 10; iter: 0; batch classifier loss: 0.250687; batch adversarial loss: 0.404790
epoch 11; iter: 0; batch classifier loss: 0.310366; batch adversarial loss:

2024-01-08 14:50:02.788655: W tensorflow/c/c_api.cc:304] Operation '{name:'04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign' id:4012 op device:{requested: '', assigned: ''} def:{{{node 04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1, 04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.688715; batch adversarial loss: 0.983039
epoch 1; iter: 0; batch classifier loss: 0.628121; batch adversarial loss: 1.100539
epoch 2; iter: 0; batch classifier loss: 0.900819; batch adversarial loss: 1.126426
epoch 3; iter: 0; batch classifier loss: 1.017392; batch adversarial loss: 1.014833
epoch 4; iter: 0; batch classifier loss: 0.997079; batch adversarial loss: 0.934127
epoch 5; iter: 0; batch classifier loss: 0.972972; batch adversarial loss: 0.840362
epoch 6; iter: 0; batch classifier loss: 1.035811; batch adversarial loss: 0.765038
epoch 7; iter: 0; batch classifier loss: 0.989932; batch adversarial loss: 0.694059
epoch 8; iter: 0; batch classifier loss: 1.016223; batch adversarial loss: 0.648634
epoch 9; iter: 0; batch classifier loss: 0.816278; batch adversarial loss: 0.583646
epoch 10; iter: 0; batch classifier loss: 0.814806; batch adversarial loss: 0.581283
epoch 11; iter: 0; batch classifier loss: 0.388503; batch adversarial loss:

### Experiment iteration 2

In [11]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 21:56:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 200,
 'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 21:56:06 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.792746; batch adversarial loss: 1.005012
epoch 1; iter: 0; batch classifier loss: 0.949620; batch adversarial loss: 1.258500
epoch 2; iter: 0; batch classifier loss: 1.028043; batch adversarial loss: 1.144422
epoch 3; iter: 0; batch classifier loss: 1.045848; batch adversarial loss: 1.075771
epoch 4; iter: 0; batch classifier loss: 0.997219; batch adversarial loss: 0.978551
epoch 5; iter: 0; batch classifier loss: 0.916257; batch adversarial loss: 0.886703
epoch 6; iter: 0; batch classifier loss: 0.807742; batch adversarial loss: 0.777476
epoch 7; iter: 0; batch classifier loss: 0.756174; batch adversarial loss: 0.765084
epoch 8; iter: 0; batch classifier loss: 0.751323; batch adversarial loss: 0.715508
epoch 9; iter: 0; batch classifier loss: 0.661528; batch adversarial loss: 0.746465
epoch 10; iter: 0; batch classifier loss: 0.532497; batch adversarial loss: 0.583311
epoch 11; iter: 0; batch classifier loss: 0.556063; batch adversarial loss:

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 19:29:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 19:29:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 19:29:01 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [11]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 21:57:34 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 400,
 'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 21:57:34 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 

...

epoch 188; iter: 0; batch classifier loss: 0.307310; batch adversarial loss: 0.543110
epoch 189; iter: 0; batch classifier loss: 0.328610; batch adversarial loss: 0.575675
epoch 190; iter: 0; batch classifier loss: 0.341410; batch adversarial loss: 0.533710
epoch 191; iter: 0; batch classifier loss: 0.313590; batch adversarial loss: 0.629138
epoch 192; iter: 0; batch classifier loss: 0.406589; batch adversarial loss: 0.609379
epoch 193; iter: 0; batch classifier loss: 0.341801; batch adversarial loss: 0.485456
epoch 194; iter: 0; batch classifier loss: 0.348638; batch adversarial loss: 0.488193
epoch 195; iter: 0; batch classifier loss: 0.364206; batch adversarial loss: 0.560961
epoch 196; iter: 0; batch classifier loss: 0.321152; batch adversarial loss: 0.530986
epoch 197; iter: 0; batch classifier loss: 0.379761; batch adversarial loss: 0.543564
epoch 198; iter: 0; batch classifier loss: 0.368707; batch adversarial loss: 0.545763
epoch 199; iter: 0; batch classifier loss: 0.275301; b

epoch 86; iter: 0; batch classifier loss: 0.484704; batch adversarial loss: 0.616500
epoch 87; iter: 0; batch classifier loss: 0.372115; batch adversarial loss: 0.571700
epoch 88; iter: 0; batch classifier loss: 0.349222; batch adversarial loss: 0.562336
epoch 89; iter: 0; batch classifier loss: 0.426290; batch adversarial loss: 0.553293
epoch 90; iter: 0; batch classifier loss: 0.394504; batch adversarial loss: 0.553478
epoch 91; iter: 0; batch classifier loss: 0.352952; batch adversarial loss: 0.571975
epoch 92; iter: 0; batch classifier loss: 0.354601; batch adversarial loss: 0.581079
epoch 93; iter: 0; batch classifier loss: 0.393039; batch adversarial loss: 0.597264
epoch 94; iter: 0; batch classifier loss: 0.392252; batch adversarial loss: 0.491050
epoch 95; iter: 0; batch classifier loss: 0.387816; batch adversarial loss: 0.544572
epoch 96; iter: 0; batch classifier loss: 0.426877; batch adversarial loss: 0.508364
epoch 97; iter: 0; batch classifier loss: 0.348239; batch adversa

epoch 182; iter: 0; batch classifier loss: 0.306418; batch adversarial loss: 0.534957
epoch 183; iter: 0; batch classifier loss: 0.379244; batch adversarial loss: 0.535264
epoch 184; iter: 0; batch classifier loss: 0.374413; batch adversarial loss: 0.535159
epoch 185; iter: 0; batch classifier loss: 0.410657; batch adversarial loss: 0.643966
epoch 186; iter: 0; batch classifier loss: 0.330432; batch adversarial loss: 0.526770
epoch 187; iter: 0; batch classifier loss: 0.340073; batch adversarial loss: 0.544341
epoch 188; iter: 0; batch classifier loss: 0.347473; batch adversarial loss: 0.527289
epoch 189; iter: 0; batch classifier loss: 0.410859; batch adversarial loss: 0.562432
epoch 190; iter: 0; batch classifier loss: 0.327870; batch adversarial loss: 0.508899
epoch 191; iter: 0; batch classifier loss: 0.326107; batch adversarial loss: 0.526721
epoch 192; iter: 0; batch classifier loss: 0.416830; batch adversarial loss: 0.587233
epoch 193; iter: 0; batch classifier loss: 0.352351; b

epoch 80; iter: 0; batch classifier loss: 0.355407; batch adversarial loss: 0.526578
epoch 81; iter: 0; batch classifier loss: 0.380903; batch adversarial loss: 0.544516
epoch 82; iter: 0; batch classifier loss: 0.388418; batch adversarial loss: 0.517333
epoch 83; iter: 0; batch classifier loss: 0.441875; batch adversarial loss: 0.489668
epoch 84; iter: 0; batch classifier loss: 0.369860; batch adversarial loss: 0.517004
epoch 85; iter: 0; batch classifier loss: 0.410424; batch adversarial loss: 0.526004
epoch 86; iter: 0; batch classifier loss: 0.421625; batch adversarial loss: 0.535275
epoch 87; iter: 0; batch classifier loss: 0.384367; batch adversarial loss: 0.571926
epoch 88; iter: 0; batch classifier loss: 0.446392; batch adversarial loss: 0.517537
epoch 89; iter: 0; batch classifier loss: 0.399745; batch adversarial loss: 0.480985
epoch 90; iter: 0; batch classifier loss: 0.373889; batch adversarial loss: 0.516888
epoch 91; iter: 0; batch classifier loss: 0.461731; batch adversa

epoch 176; iter: 0; batch classifier loss: 0.445722; batch adversarial loss: 0.607869
epoch 177; iter: 0; batch classifier loss: 0.337475; batch adversarial loss: 0.516860
epoch 178; iter: 0; batch classifier loss: 0.342766; batch adversarial loss: 0.471407
epoch 179; iter: 0; batch classifier loss: 0.377399; batch adversarial loss: 0.562969
epoch 180; iter: 0; batch classifier loss: 0.362587; batch adversarial loss: 0.507819
epoch 181; iter: 0; batch classifier loss: 0.350906; batch adversarial loss: 0.535367
epoch 182; iter: 0; batch classifier loss: 0.360282; batch adversarial loss: 0.526372
epoch 183; iter: 0; batch classifier loss: 0.387501; batch adversarial loss: 0.635742
epoch 184; iter: 0; batch classifier loss: 0.432212; batch adversarial loss: 0.635232
epoch 185; iter: 0; batch classifier loss: 0.362908; batch adversarial loss: 0.590154
epoch 186; iter: 0; batch classifier loss: 0.343385; batch adversarial loss: 0.544544
epoch 187; iter: 0; batch classifier loss: 0.396747; b

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 03:55:49 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 500,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 03:55:49 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 03:55:49 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [13]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 04:49:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 600,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 04:49:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 04:49:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]